In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Meta-analyses of Benefits and Harms by Surgical Intervention

## Key Question 4
**Do benefits and harms differ by surgical technique?**

The approach to estimating the relative effects of surgical techniques on benefits (*i.e.* utilization) and harms mirror that for pharmacological interventions. That is, modeling count outcomes (*e.g.* occurrences of harms) as Poisson or binomial random variables and continuous outcomes as normal random variables, and estimating the effect sizes associated with each procedure.

The effect size model should include and indicator for indication (Obstructive sleep apnea (OSA), tonsillectomy, or mixed), as this is anticipated to influence outcomes. Relevant outcomes include time to return to normal diet, use of post-operative medications, and bleeding outcomes (primary, secondary, readmission due to bleeding, re-operation due to bleeding).


## Key Question 3 

**Do benefits and harms differ between partial tonsillectomy and total tonsillectomy?**

The relative effect of full compared to partial tonsillectomy may be evaluated using an indicator variable for full tonsillectomy relative to a baseline effect of partial tonsillectomy (or the reverse, depending on the prevalence of each intervention in the available data). Depending on whether counts of harms or continuous differences in diet return times are being modeled as the response, this effect can be modeled as:

$$\theta_{ki} = g(\beta_0 + \beta_1 I(x^{(full)}_{ki}))$$

where $I$ is the indicator function that evaluates to one if $x^{(full)}_{ki}$ is true (indicating full tonsillectomy) and zero otherwise, and $g$ is an appropriate transformation, depending on the type of outcome (*e.g.* inverse-logit if we are modeling binomial counts of harms). As with KQ4, indication should also be included as a covariate. As with KQ 3, outcomes of interest include time to return to normal diet, use of post-operative medications, and bleeding outcomes (primary, secondary, readmission due to bleeding, re-operation due to bleeding).

An efficient approach to addressing this question may be to use a common model for KQ 3 and 4, with partial vs total removal simply included as a covariate in the KQ 3 model. Potentially, this would allow for comparisons of surgical technique within the partial/total subpopulation, using explicit interaction effects in the model.

## Import and Clean Data

In [95]:
outcomes = pd.read_excel('data/Tonsillectomy_Master_Outcome_data_Comparative_studies_04112016.xlsx', 
              sheetname='All_outcomes_KQ3-6', 
              na_values=['null', 'ND', 'NR'])

In [96]:
outcomes.head(3)

,Citation,REFID,Child Paper,Number of Arms,Rx Grouping,Group_Desc,Intervention_Category,Dose,Route,Rx_Durn,...,Outcome _Q3,Outcome Min,Outcome Max,Outcome 95% L,Outcome 95% H,Other stats Name,Other Stats,Results,Comments,Comments 2
0,K. W. Chang. Intracapsular versus subcapsular...,2428,NaN,2,G1:,Intracapsular coblation tonsillectomy,partial coblation,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Outcome Mean %,56,"G1 vs. G2, p = 0.0549","return to normal diet data presented as ""mean ...",NaN
1,K. W. Chang. Intracapsular versus subcapsular...,2428,NaN,2,G2:,Subcapsular coblation tonsillectomy,total coblation,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Outcome Mean %,42,NaN,NaN,NaN
2,K. W. Chang. Intracapsular versus subcapsular...,2428,NaN,2,G1:,Intracapsular coblation tonsillectomy,partial coblation,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Outcome Mean %,73,"G1 vs. G2, p < 0.05","return to normal activity reported as ""mean pe...",NaN


In [97]:
study_info = pd.read_excel('data/Tonsillectomy_Master_Outcome_data_Comparative_studies_04112016.xlsx', 
              sheetname='BASIC_Charact', 
              na_values=['null', 'ND'])

In [98]:
study_info.head()

,REFID,Child_ID,Population_Catg,Diagnostic Method,Population specify,Trial name,Study Design,Rx setting,Intervention TMPT,Rx Durn,...,N_Rx_Completed,N1,N2,N3,N4,N5,Age(years) Mean ±SD/ range,Antibiotics given at any time,Comments,Comments 2
0,806 _534_359_164_195_120,NaN,OSDB,PSG,obstructive sleep apnea syndrome without prolo...,CHAT,RCT,Sleep centers,surgery,NaN,...,397,194.0,203.0,NaN,NaN,NaN,5 to 9.9,No,"subgroup analysis by race, obesity",NaN
1,3241,NaN,OSDB,PSG,Children with OSA,NaN,RCT,Pediatric otolaryngology private office and cl...,surgery,NaN,...,41,21.0,11.0,9.0,NaN,NaN,2 to 14,No,Mean age of the children who completed the pro...,NaN
2,546,NaN,OSDB,PSG,Children with clinically suspected SDB and hea...,NaN,Prospective cohort,recruited from previous SDB study on blood pre...,surgery,NaN,...,38,12.0,26.0,NaN,NaN,NaN,7 to 12,No,Did not extract data for control arm in the ou...,NaN
3,2294,NaN,OSDB,videonasopharyngoscopic study with pharyngeal ...,Pediatric patients with pharyngeal and palatin...,NaN,Prospective cohort,Otorhinolaryngology public service,surgery,NaN,...,22,12.0,10.0,NaN,NaN,NaN,3y4m to 10y3m,No,G1: 3 years 4 month to 9 years 8 month; G2: 4 ...,NaN
4,1729,NaN,OSDB,PSG,Children with PSG-proven OSA,NaN,Prospective cohort,Sleep-wake disorder unit,surgery,NaN,...,20,14.0,6.0,NaN,NaN,NaN,G1: 6.4±2.5 G2: 5.4±2.3,No,NaN,NaN


In [99]:
outcomes.columns

Index(['Citation', 'REFID', 'Child Paper', 'Number of \nArms', 'Rx Grouping',
       'Group_Desc', 'Intervention_Category', 'Dose', 'Route', 'Rx_Durn',
       'Last Assesment tmpt for the study', 'Followup duration category',
       'OUTC_Main_\nCATG', 'Outc_SUB_\nCATG', 'Outcome_specify',
       'Presentation \nlocation', 'Outc_Unit', 'Outc_Tool',
       'Outcome timepoint (when was this outcome measured, e.g., in PACU, 12 months post-op, immediately post-op--would need a row for each outcome at each timepoint of interest)',
       'Outcome sample size', 'Outcome\ncount', 'Outcome  \n%',
       'Outcome\n Mean', 'Outcome \nSD', 'Outcome \n SE', 'Outcome \n Median',
       'Outcome\n _Q1', 'Outcome \n_Q3', 'Outcome\n  Min', 'Outcome \n Max',
       'Outcome \n 95% L', 'Outcome \n 95% H', 'Other stats \nName',
       'Other Stats', 'Results', 'Comments', 'Comments 2'],
      dtype='object')

In [139]:
data_merged = (outcomes.merge(study_info, on='REFID')[['REFID', 
                                                      'Intervention_Category', 
                                                      'OUTC_Main_\nCATG', 
                                                      'Outc_SUB_\nCATG',
                                                      'Outcome_specify',
                                                      'Study Design', 
                                                      'Population_Catg', 
                                                      'Outcome sample size',
                                                      'Outcome\ncount',
                                                      'Outcome\n Mean', 
                                                      'Outcome \nSD',
                                                      'Outc_Unit']]
               .rename(columns={'OUTC_Main_\nCATG':'Outcome_cat', 
                                'Outc_SUB_\nCATG':'Outcome_subcat',
                                'Outcome\ncount':'Outcome_count',
                                'Outcome\n Mean':'Outcome_mean',
                                'Outcome \nSD':'Outcome_sd',
                                'Outcome sample size':'Outcome_n'}))
data_merged.head()

,REFID,Intervention_Category,Outcome_cat,Outcome_subcat,Outcome_specify,Study Design,Population_Catg,Outcome_n,Outcome_count,Outcome_mean,Outcome_sd,Outc_Unit
0,2428,partial coblation,Return to normal diet,Time to Return to normal diet,return to normal diet (mean % of diet resumed),RCT,OSDB,34.0,NaN,NaN,NaN,NaN
1,2428,total coblation,Return to normal diet,Time to Return to normal diet,return to normal diet (mean % of diet resumed),RCT,OSDB,35.0,NaN,NaN,NaN,NaN
2,2428,partial coblation,Return to normal diet,Time to Return to normal diet,return to normal diet (mean % of diet resumed),RCT,OSDB,34.0,NaN,NaN,NaN,NaN
3,2428,total coblation,Return to normal diet,Time to Return to normal diet,return to normal diet (mean % of diet resumed),RCT,OSDB,35.0,NaN,NaN,NaN,NaN
4,2428,partial coblation,Return to normal activity,Time to Return to normal activity,return to normal activity (mean % of activity ...,RCT,OSDB,34.0,NaN,NaN,NaN,NaN


In [143]:
data_merged.groupby(['Study Design', 'Intervention_Category'])['Outcome_n'].sum().astype(int)

Study Design          Intervention_Category                                 
Non-randomized trial  partial coblation                                          838
                      perioperative steroid                                      248
                      placebo                                                    232
                      postoperative analgesic                                    450
                      postoperative nsaid                                        230
                      total bilateral                                             90
                      total coblation                                             42
                      total cold                                                1998
                      total electrocautery                                      1272
                      total thermal welding                                      416
                      total unilateral                                   

In [144]:
data_merged['Intervention_Category'] = data_merged.Intervention_Category.str.lower().str.strip()
clean_data = data_merged.replace({'Intervention_Category':{'saline':'control',
                        'control (no rx)':'control',
                        'control (no dexamethasone)':'control',
                        'no antibiotics':'control',
                        'no steroid':'control',
                        'placebo':'control'}})

In [145]:
surgery_data = clean_data[clean_data.Intervention_Category.str.contains('total') 
                   | clean_data.Intervention_Category.str.contains('partial')].copy()

Indicator variable for partial tonsillectomy

In [146]:
surgery_data['partial'] = surgery_data.Intervention_Category.str.contains('partial')

Surgical technique

In [147]:
surgery_data['technique'] = (surgery_data.Intervention_Category
        .apply(lambda x: ' '.join(x.split(' ')[1:]))
        .replace({'tonsillectomy - coblation':'coblation'}))

surgery_data = surgery_data[~(surgery_data.technique.str.contains('specified')|
                             surgery_data.technique.str.contains('\+'))]

In [148]:
surgery_data.technique.value_counts()

cold                   135
electrocautery         134
coblation              113
harmonic scalpel        30
laser                   21
molecular resonance     19
microdebrider           16
thermal welding         13
bilateral                1
unilateral               1
Name: technique, dtype: int64

Drop poorly-represented methods

In [149]:
technique_includes = surgery_data.technique.value_counts().index[:-2].tolist()
surgery_data = surgery_data[surgery_data.technique.isin(technique_includes)]

In [150]:
technique_includes

['cold',
 'electrocautery',
 'coblation',
 'harmonic scalpel',
 'laser',
 'molecular resonance',
 'microdebrider',
 'thermal welding']

In [151]:
surgery_data.Outcome_cat.value_counts()

Harms                        306
Return to normal diet         74
Return to normal activity     50
Throat infection              10
Health                        10
Tonsillar regrowth             8
Sleep                          8
Pain management                7
Return to School               4
Cognitive                      2
QOL                            2
Name: Outcome_cat, dtype: int64

Pool throat infection with harms

In [152]:
surgery_data = surgery_data.replace({'Outcome_cat':{'Throat infection':'Harms'}})

In [153]:
surgery_data[surgery_data.Outcome_cat=='Harms'].Outcome_subcat.value_counts()

Harms: Post-op bleeding                150
Harms-Re-operation for bleeding         49
Harms- readmission for dehydration      34
Harms- readmission for bleeding         27
Harms- readmission for PONV             11
Harms-Death-30 day                      10
Throat infection-number                  8
Harms- readmission for post-op pain      6
Harms-other                              6
Harms- ER visit for post-op pain         3
Harms- ER visit-Unspecified              2
Harms- ER visit for bleeding             2
Throat infection-# strep infections      2
Harms-Hospital admission                 2
Harms-Velopharyngeal insufficiency       2
Harms - readmission for dehydration      2
Name: Outcome_subcat, dtype: int64

Subset of return to normal diet/activity

In [154]:
return_subset = surgery_data[surgery_data.Outcome_cat.str.startswith('Return to normal') &
                            (surgery_data.Outc_Unit=='days')]

In [155]:
dummies = pd.get_dummies(return_subset.Outcome_cat).T
outcome_means = (dummies * return_subset.Outcome_mean).T
outcome_means.columns = 'activity_mean', 'diet_mean'
outcome_sds = (dummies * return_subset.Outcome_sd).T.dropna()
outcome_sds.columns = 'activity_sd', 'diet_sd'

In [166]:
return_outcomes = pd.merge(outcome_means, outcome_sds, left_index=True, right_index=True)

In [167]:
merged_data = surgery_data.merge(return_outcomes, left_index=True, right_index=True)

Summary of data by technique

In [178]:
np.mean?

In [179]:
(merged_data
         .groupby(['REFID', 'technique'])
        .agg({'Outcome_n': lambda x: int(sum(x)), 
              'activity_mean': lambda x: np.round(np.mean(x), 1), 
              'diet_mean': lambda x: np.round(np.mean(x), 1)}))

Outcome_n  diet_mean  activity_mean
REFID technique                                           
953   cold                 101.0        5.4            0.0
1372  cold                  99.0        7.0            0.0
      electrocautery       102.0        9.3            0.0
      thermal welding      104.0        7.3            0.0
2198  coblation             92.0        2.4            2.4
      electrocautery        94.0        3.2            3.3
      microdebrider         98.0        2.3            2.3
2891  electrocautery        76.0        2.2            1.9
      microdebrider         72.0        1.4            1.2
3002  cold                   8.0        2.8            0.0
      electrocautery        23.0        2.6            0.0
3253  coblation             88.0        2.3            3.7
      electrocautery        90.0        2.6            3.4
6317  coblation             94.0        3.1            3.8
      cold                  94.0        4.6            5.8

Summary of data by partial vs total removal

In [180]:
(merged_data
         .groupby(['REFID', 'partial'])
        .agg({'Outcome_n': lambda x: int(sum(x)), 
              'activity_mean': lambda x: np.round(np.mean(x), 1), 
              'diet_mean': lambda x: np.round(np.mean(x), 1)}))

Outcome_n  diet_mean  activity_mean
REFID partial                                     
953   False         51.0        7.1            0.0
      True          50.0        3.8            0.0
1372  False        305.0        7.9            0.0
2198  False        284.0        2.6            2.6
2891  False         76.0        2.2            1.9
      True          72.0        1.4            1.2
3002  False         31.0        2.7            0.0
3253  False        178.0        2.4            3.5
6317  False        188.0        3.9            4.8

### Model Specification

In [17]:
from pymc3 import (NUTS, sample, Model, Deterministic, find_MAP,
                  Binomial, Normal, HalfCauchy, advi,
                  traceplot, summary, forestplot)
import theano.tensor as tt

def tinvlogit(x):
    return tt.exp(x) / (1 + tt.exp(x))

Couldn't import dot_parser, loading of dot files will not be possible.
